# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
load_csv = "WeatherPyData.csv"
weather_df = pd.read_csv(load_csv)
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Lavrentiya,65.5833,-171.0000,7.30,96,100,8.95,RU,1645582506
1,Hilo,19.7297,-155.0900,75.25,81,100,1.01,US,1645582502
2,Jamestown,42.0970,-79.2353,56.30,72,100,11.50,US,1645582522
3,Port Hedland,-20.3167,118.5667,95.05,59,100,12.66,AU,1645582544
4,Hermanus,-34.4187,19.2345,62.80,73,0,3.71,ZA,1645582496


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
gmaps.configure(api_key=g_key)

In [6]:
#Store latitude and longitude data
lat_lon = weather_df[['Lat', 'Lng']]

#Store humidity data
humidity = weather_df["Humidity"]

In [7]:
#Plot Heatmap!

figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Use gmaps.figure to make figure displayed properly 

fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

#Add heat layer to map 
#For max intensity in the heat map, try setting it to the highest humidity found in the data set.

heat_layer = gmaps.heatmap_layer(lat_lon, weights=humidity,dissipating=False, max_intensity=100, point_radius=3)

# Add heat layer

fig.add_layer(heat_layer)

# Display figure

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
fitting_weather = weather_df.loc[(
    (weather_df["Cloudiness"]> 70) &
    (weather_df["Max Temp"] <= 65) &
    (weather_df["Max Temp"] <= 80) &
    (weather_df["Country"] != 'US')& 
    (weather_df["Country"] != 'RU')
    )]
fitting_weather = fitting_weather.dropna()
#y_weather = my_weather.reset_index()
fitting_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
19,Atyrau,47.1167,51.8833,32.02,100,100,6.71,KZ,1645582637
30,Punta Arenas,-53.1500,-70.9167,51.91,82,75,8.05,CL,1645582490
36,Upernavik,72.7868,-56.1549,-12.33,96,100,7.92,GL,1645582503
38,Yaypan,40.3758,70.8156,45.16,49,100,2.39,UZ,1645582641
41,Norman Wells,65.2820,-126.8329,8.64,66,100,2.30,CA,1645582533
...,...,...,...,...,...,...,...,...,...
530,Zinder,13.8072,8.9881,63.81,10,99,9.13,NE,1645582804
534,Ofakim,31.3141,34.6203,53.74,55,85,6.44,IL,1645582806
535,Cumbernauld,55.9469,-3.9905,43.03,84,97,20.56,GB,1645582806
537,Emet,39.3430,29.2585,43.07,86,100,4.09,TR,1645582807


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df = fitting_weather
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
19,Atyrau,47.1167,51.8833,32.02,100,100,6.71,KZ,1645582637
30,Punta Arenas,-53.1500,-70.9167,51.91,82,75,8.05,CL,1645582490
36,Upernavik,72.7868,-56.1549,-12.33,96,100,7.92,GL,1645582503
38,Yaypan,40.3758,70.8156,45.16,49,100,2.39,UZ,1645582641
41,Norman Wells,65.2820,-126.8329,8.64,66,100,2.30,CA,1645582533
...,...,...,...,...,...,...,...,...,...
530,Zinder,13.8072,8.9881,63.81,10,99,9.13,NE,1645582804
534,Ofakim,31.3141,34.6203,53.74,55,85,6.44,IL,1645582806
535,Cumbernauld,55.9469,-3.9905,43.03,84,97,20.56,GB,1645582806
537,Emet,39.3430,29.2585,43.07,86,100,4.09,TR,1645582807


In [10]:
#lat_lngs=zip(lat,lng)
hotels = []
lat=hotel_df["Lat"]
lng=hotel_df["Lng"]
lat_lngs=zip(lat,lng)

In [11]:
url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
type="lodging"
bounds=5000

for lat_lng in lat_lngs:
    coord=f"{lat_lng[0]},{lat_lng[1]}"
    params={"location": coord,"radius":bounds,"types": type,"key":g_key}
    response=requests.get(url,params=params)
    hotel_data=response.json()
    try:
        hotels.append(hotel_data["results"][1]["name"])
    except:
        print("no hotels found...")
        hotels.append("no hotels found...")

no hotels found...
no hotels found...
no hotels found...
no hotels found...
no hotels found...
no hotels found...
no hotels found...
no hotels found...
no hotels found...
no hotels found...


In [12]:
hotels

['Hotel Atyrau',
 'Hotel Cabo de Hornos',
 'Café de Upernavik',
 'Nabiyevlar oilasi',
 'Yamouri Inn',
 'Scandic Karasjok',
 'Tuktu B&B',
 'Praca Våler dom',
 'no hotels found...',
 'Bikers Paradise',
 'Hotel Playa El Cable',
 'Höfn Inn Guesthouse',
 'Hosteria Millaneco',
 'no hotels found...',
 'no hotels found...',
 'no hotels found...',
 'Wulingcheng Hotel',
 'Hotel Angmagssalik',
 'Haoting International Hotel',
 'Geilin Bed & Breakfast',
 "Seria's Hjem",
 'no hotels found...',
 'Waatazuma Guest House',
 'Koket',
 'ibis Hong Kong Central & Sheung Wan',
 'Graciosa Resort - Biosphere Island Hotel',
 'Letrip Hospedaria e Hambúrgueres',
 'Lyuks Gostinitsa',
 'Canadas Best Value Inn River View Hotel',
 'Hotel restaurant The Island of Margarita',
 'Narsaq Kayak Hostel',
 'Ferienwohnungen Karin Zeiler',
 'Wanning Hotel',
 'Mendi Travellers inn',
 'Bergstadens Hotel',
 'Auyuittuq Lodge (Inns North)',
 'Bar B Motel',
 'Aska Hostel',
 'Jinjiang Metropolo Hotel Xuzhou Feng County Jinjiang Mansi

In [13]:
#add the new column
hotel_df["Hotel Name"]= hotels
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
19,Atyrau,47.1167,51.8833,32.02,100,100,6.71,KZ,1645582637,Hotel Atyrau
30,Punta Arenas,-53.1500,-70.9167,51.91,82,75,8.05,CL,1645582490,Hotel Cabo de Hornos
36,Upernavik,72.7868,-56.1549,-12.33,96,100,7.92,GL,1645582503,Café de Upernavik
38,Yaypan,40.3758,70.8156,45.16,49,100,2.39,UZ,1645582641,Nabiyevlar oilasi
41,Norman Wells,65.2820,-126.8329,8.64,66,100,2.30,CA,1645582533,Yamouri Inn
...,...,...,...,...,...,...,...,...,...,...
530,Zinder,13.8072,8.9881,63.81,10,99,9.13,NE,1645582804,hôtel central
534,Ofakim,31.3141,34.6203,53.74,55,85,6.44,IL,1645582806,Another route
535,Cumbernauld,55.9469,-3.9905,43.03,84,97,20.56,GB,1645582806,Premier Inn Glasgow (Cumbernauld) hotel
537,Emet,39.3430,29.2585,43.07,86,100,4.09,TR,1645582807,EMET ŞAHANLAR OTEL & GIDA PAZARI


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# Add marker layer ontop of heat map
hotel_fig=gmaps.figure()
markers=gmaps.marker_layer(locations)
marker_layer=gmaps.marker_layer(locations)

# Display figure
hotel_fig.add_layer(marker_layer)
hotel_fig

Figure(layout=FigureLayout(height='420px'))